## 인코더-디코더

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

In [10]:
input_data = np.array([[[1, 2, 3]]], dtype=np.float32)
output_data = np.array([[[2, 3, 4]]], dtype=np.float32)

# 입력 데이터 형상 변경
input_data = input_data[:, :, :1]

In [11]:
#인코더 정의
encoder_inputs = Input(shape=(None, 1))
encoder_lstm = LSTM(units = 16, return_state = True)
_, encoder_h, encoder_c = encoder_lstm(encoder_inputs)
encoder_states = [encoder_h, encoder_c]

In [12]:
#디코더 정의
decoder_inputs = Input(shape=(None,1))
decoder_lstm = LSTM(units = 16, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(units=1, activation = 'linear')
decoder_outputs = decoder_dense(decoder_outputs)

In [13]:
model = Model(inputs = [encoder_inputs, decoder_inputs], outputs = decoder_outputs)

In [14]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit([input_data, input_data], output_data, batch_size = 1, epochs = 100)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 9.9420
Epoch 2/100
1/1 [==============================] - 0s 7ms/step - loss: 9.9028
Epoch 3/100
1/1 [==============================] - 0s 5ms/step - loss: 9.8638
Epoch 4/100
1/1 [==============================] - 0s 7ms/step - loss: 9.8249
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 9.7862
Epoch 6/100
1/1 [==============================] - 0s 6ms/step - loss: 9.7476
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 9.7092
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 9.6709
Epoch 9/100
1/1 [==============================] - 0s 6ms/step - loss: 9.6327
Epoch 10/100
1/1 [==============================] - 0s 7ms/step - loss: 9.5946
Epoch 11/100
1/1 [==============================] - 0s 6ms/step - loss: 9.5567
Epoch 12/100
1/1 [==============================] - 0s 6ms/step - loss: 9.5188
Epoch 13/100
1/1 [==============================] - 0s 6ms/ste

## Word2Vec 실습

In [22]:
!pip install lxml
!pip install nltk

     ---------------------------------------- 1.5/1.5 MB 24.2 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 97.9/97.9 kB 5.8 MB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
     ---------------------------------------- 270.2/270.2 kB ? eta 0:00:00


In [23]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [24]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x1ab9ede24c8>)

In [26]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tnv59\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [27]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [28]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [29]:
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


In [31]:
!pip install --upgrade gensim

     --------------------------------------- 24.0/24.0 MB 43.7 MB/s eta 0:00:00
     ------------------------------------- 983.8/983.8 kB 31.4 MB/s eta 0:00:00
     ---------------------------------------- 57.0/57.0 kB ? eta 0:00:00


In [32]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [33]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8343969583511353), ('guy', 0.803366482257843), ('lady', 0.7468178868293762), ('boy', 0.7433486580848694), ('girl', 0.7384998798370361), ('gentleman', 0.7176277041435242), ('soldier', 0.7077347636222839), ('kid', 0.6815178394317627), ('friend', 0.6570190191268921), ('poet', 0.6566721796989441)]


In [34]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [35]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8343969583511353), ('guy', 0.803366482257843), ('lady', 0.7468178868293762), ('boy', 0.7433486580848694), ('girl', 0.7384998798370361), ('gentleman', 0.7176277041435242), ('soldier', 0.7077347636222839), ('kid', 0.6815178394317627), ('friend', 0.6570190191268921), ('poet', 0.6566721796989441)]


## Glove 실습

In [15]:
!pip install glove_python_binary

     -------------------------------------- 225.8/225.8 kB 7.0 MB/s eta 0:00:00
     --------------------------------------- 34.1/34.1 MB 43.5 MB/s eta 0:00:00


In [16]:
from glove import Corpus, Glove

corpus = Corpus()

In [37]:
corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [38]:
print(glove.most_similar("man"))

[('woman', 0.9602903226405134), ('guy', 0.886511040965002), ('girl', 0.8601827685608269), ('young', 0.8368183895002479)]


In [39]:
print(glove.most_similar("boy"))

[('girl', 0.9406638550554005), ('kid', 0.8461361809953235), ('man', 0.8351529055028157), ('woman', 0.8341594103767832)]


In [40]:
print(glove.most_similar("university"))

[('harvard', 0.8894672345057508), ('mit', 0.8587181620366068), ('stanford', 0.840225345859485), ('cambridge', 0.8302451990883875)]


In [41]:
print(glove.most_similar("water"))

[('clean', 0.8376948203313356), ('fresh', 0.8318064084404437), ('air', 0.825492904192858), ('food', 0.8181621596765289)]
